In [ ]:
import tensorflow as tf
import matploitlib.pyplot as plt 
import numpy as np


# Ruta raiz
PATH = "/"

# Ruta de datos de entrada 
INPATH =  PATH + '/inputFlowers'
# Ruta de datos de salida
OUTPATH = PATH + '/targetFlowers'
# Ruta de los checkpoints
CKPATH = PATH + '/checkpoints'